In [ ]:
from datasets import load_dataset

In [ ]:
dataset=load_dataset('csv',data_files="/content/formatted_headline_data.csv",split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset[0]

{'headlines': 'High School Students Create Haunting Artwork About Standardized Tests',
 'sarcastic_headlines': 'High school students create haunting artwork about standardized tests because clearly, their futures depend on it.',
 'text': 'You are a savage, disrespectful and witty paraphrasing tool. You rephrase a news headline into a funny, creatively sarcastic way. Write a sarcastic headline by rephrasing the input headline.\n\n### Headline:\nHigh School Students Create Haunting Artwork About Standardized Tests\n\n### Sarcastic Headline:\nHigh school students create haunting artwork about standardized tests because clearly, their futures depend on it.'}

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers==0.0.27" --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-jv1mpntj/unsloth_12119307503249dcab4bb84ffc78e264
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-jv1mpntj/unsloth_12119307503249dcab4bb84ffc78e264
  Resolved https://github.com/unslothai/unsloth.git to commit 2eb6b0d5f363a60ed3792ea1f04250537ac66939
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 148.9 MB/s eta 0:00:00

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 MB 6.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.12.8: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Phi-3 and Llama often have missing pad tokens.
# We set it to the unknown token or eos token to prevent errors.
if tokenizer.pad_token is None:
    tokenizer.pad_token
tokenizer.padding_side = "right"

In [ ]:
from unsloth.chat_templates import get_chat_template

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
)

In [ ]:
from datasets import load_dataset
# Replace 'sarcasm_data.csv' with your actual filename
dataset = load_dataset("csv", data_files = "/content/formatted_headline_data.csv", split = "train")

# 3. Define the System Prompt
system_prompt = "You are a savage, disrespectful and witty paraphrasing tool. You rephrase a news headline into a funny, creatively sarcastic way."

In [ ]:
def formatting_prompts_func(examples):
    inputs = examples["headlines"]
    outputs = examples["sarcastic_headlines"]

    texts = []
    for input_text, output_text in zip(inputs, outputs):
        # This explicitly structures the conversation: System -> User -> Assistant
        conversation = [
            {"from": "system", "value": system_prompt},
            {"from": "human",  "value": f"Write a sarcastic headline by rephrasing the input headline.\n\n### Headline:\n{input_text}"},
            {"from": "gpt",    "value": output_text}
        ]

        # Apply the chat template
        # This automatically adds "<|user|>", "<|assistant|>", and critically "<|end|>" (EOS)
        text = tokenizer.apply_chat_template(
            conversation,
            tokenize = False,
            add_generation_prompt = False
        )
        texts.append(text)

    return { "text" : texts }

In [ ]:
# 4. Apply Formatting
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
print(dataset[0]["text"])

<|system|>
You are a savage, disrespectful and witty paraphrasing tool. You rephrase a news headline into a funny, creatively sarcastic way.<|end|>
<|user|>
Write a sarcastic headline by rephrasing the input headline.

### Headline:
High School Students Create Haunting Artwork About Standardized Tests<|end|>
<|assistant|>
High school students create haunting artwork about standardized tests because clearly, their futures depend on it.<|end|>



In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        max_steps = 200,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,

        # CHECKPOINTING
        save_strategy = "steps",
        save_steps = 20,
        save_total_limit = 2,

        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/content/drive/My Drive/Sarcastic/checkpoints",
    ),
)

trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,100 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Step,Training Loss
1,2.873200
2,2.735000
3,2.737900
4,2.818900
5,2.860600
6,2.750000
7,2.860500
8,2.734300
9,2.701200
10,2.707700


Step,Training Loss
1,2.873200
2,2.735000
3,2.737900
4,2.818900
5,2.860600
6,2.750000
7,2.860500
8,2.734300
9,2.701200
10,2.707700


Unsloth: Will smartly offload gradients to save VRAM!


train/epoch,▁▁▂▂▂▂▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▁▁▁▁▂▂▂▂▂▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,█▆▇▁▁█▆▅▄▄▇▆▆▆▅▁▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▇██▇▇▂▂▁▁▁▂▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/loss,██▇▂▂▂▂▁▁▁██▃▃▂▁▂▂▁▁▂▁▁▁▁▁▁▂▁▂▁▂▁▁▁▁▂▂▂▁
total_flos,4369578474332160.0
train/epoch,0.7619
train/global_step,200
train/grad_norm,0.24271
train/learning_rate,0.0
train/loss,0.8367


In [ ]:
# Saving Lora Apdapter to drive
drive_path = "/content/drive/MyDrive/Sarcastic/models"

# 3. Save the LoRA adapters
model.save_pretrained(drive_path)

# 4. Save the tokenizer (Important to ensure decoding works later!)
tokenizer.save_pretrained(drive_path)

('/content/drive/MyDrive/Sarcastic/models/tokenizer_config.json',
 '/content/drive/MyDrive/Sarcastic/models/special_tokens_map.json',
 '/content/drive/MyDrive/Sarcastic/models/chat_template.jinja',
 '/content/drive/MyDrive/Sarcastic/models/tokenizer.model',
 '/content/drive/MyDrive/Sarcastic/models/added_tokens.json',
 '/content/drive/MyDrive/Sarcastic/models/tokenizer.json')

In [ ]:
from huggingface_hub import login
login()

In [ ]:


# 2. Merge and Push to Hub
# We use "merged_16bit" because it is the most compatible with the Serverless API.
model.push_to_hub_merged(
    "hg-goswami/sarcastic-phi3", # Replace with your HF username and model name
    tokenizer,
    save_method = "merged_16bit",
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...stic-phi3/tokenizer.model: 100%|##########|  500kB /  500kB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   1%|          | 25.1MB / 2.65GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [07:03<00:00, 211.78s/it]


Unsloth: Merge process complete. Saved to `/content/hg-goswami/sarcastic-phi3`


In [ ]:
# Define your topics/questions
topics = [
    "Traditionally, the word, high command, is associated with the Congress and the Nehru-Gandhi family. But in a twist in the tale, the contemporary high command has shifted location from Congress to the BJP.",
    "Virat Kohli has returned to the nets as he steps up preparations for the Vijay Hazare Trophy. The star batter was seen sweating it out in training and is set to turn out for Delhi under captain Rishabh Pant, with an eye on form ahead of the next ODI series.",
    "Veteran Congress leader P. Chidambaram on Sunday urged the Union government to stop “vindictive action” against party leaders Sonia Gandhi and Rahul Gandhi in connection with the National Herald case, following Special judge Vishal Gogne of the Rouse Avenue Court rejecting the Directorate of Enforcement (ED) money laundering complaint."
]

# Enable Inference Mode
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)

print("🤖 Asking the model... (This may take a moment)\n")
print("="*60) # A nice separation line at the start

for question in topics:
    messages = [
        {"from": "system", "value": "You are a witty and sarcastic news assistant."},
        {"from": "human", "value": question}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(
        input_ids = inputs,
        max_new_tokens = 128,
        use_cache = True,
        temperature = 0.7,
    )

    answer_only = tokenizer.batch_decode(outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0]


    print(f"🔹 QUESTION: {question}")
    print(f"🔸 ANSWER:   {answer_only.strip()}")
    print("-" * 60)
    print("\n")

🤖 Asking the model... (This may take a moment)

🔹 QUESTION: Traditionally, the word, high command, is associated with the Congress and the Nehru-Gandhi family. But in a twist in the tale, the contemporary high command has shifted location from Congress to the BJP.
🔸 ANSWER:   Ah, the ever-evolving landscape of Indian politics, where the high command, once a bastion of the Congress party and the Nehru-Gandhi dynasty, has undergenoy a rather dramatic shift. It seems the Bharatiya Janata Party (BJP) has taken the reins, steering the ship of power in a new direction.

This change in the high command reflects the dynamic nature of Indian politics, where power ebbs and flows, and alliances are constantly being forged and broken. The BJP's rise to promin
------------------------------------------------------------


🔹 QUESTION: Virat Kohli has returned to the nets as he steps up preparations for the Vijay Hazare Trophy. The star batter was seen sweating it out in training and is set to turn o